## Geometry

We want to model a quarter of a plate with an elliptical hole. The domain itself is represented by collocation points, the boundaries are represented by uniformly sampled points along the perimeter.

In [2]:
import torch
from torch.optim.lr_scheduler import StepLR, ExponentialLR
import numpy as np
from scipy.stats import qmc
from plotly.express.colors import sequential
import plotly.graph_objects as go
import plotly.figure_factory as ff
from tqdm import tqdm
import wandb
import matplotlib.pyplot as plt

torch.set_default_dtype(torch.float64)
from global_constants import L, MU, LBD

# Number of collocation points
N = 25

B0 = 0
EPS0 = 0.1

# Epochs
epochs = 5000
# Batch size
batch_size = 64
# Leanring rate
lr = 0.01 #normally 0.001
# Scheduler step width
scheduler_step = 150 #normally 2000
# Gamma factor of scheduler
scheduler_gamma = 0.7

# Number of hidden neurons
hn = 50

# Weight of PDE loss
W_PDE = 0.01

# Create collocation points
points = L * qmc.LatinHypercube(d=2).random(N**2)
collocation = torch.tensor(points, requires_grad=True).double()

# Boundary points
x_top = torch.linspace(0, L, N, requires_grad=True).double()
y_top = L * torch.ones((N, 1), requires_grad=True).double()
top = torch.column_stack([x_top, y_top]).double()

x_bottom = torch.linspace(0, L, N).double()
y_bottom = torch.zeros((N, 1)).double()
bottom = torch.column_stack([x_bottom, y_bottom]).double()

x_left = torch.zeros((N, 1)).double()
y_left = torch.linspace(0, L, N).double()
left = torch.column_stack([x_left, y_left]).double()

x_right = L * torch.ones((N, 1)).double()
y_right = torch.linspace(0, L, N).double()
right = torch.column_stack([x_right, y_right]).double()


# Visualize geometry
with torch.no_grad():
    mode = "markers"
    gray = dict(color="#C9C5BC")
    green = dict(color="#006561")
    black = dict(color="black")
    orange = dict(color="#ff7f0e")
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=collocation[:, 0], y=collocation[:, 1], mode=mode, marker=gray))
    fig.add_trace(go.Scatter(x=top[:, 0], y=top[:, 1], mode=mode, marker=black))
    fig.add_trace(go.Scatter(x=bottom[:, 0], y=bottom[:, 1], mode=mode, marker=orange))
    fig.add_trace(go.Scatter(x=left[:, 0], y=left[:, 1], mode=mode, marker=green))
    fig.add_trace(go.Scatter(x=right[:, 0], y=right[:, 1], mode=mode, marker=green))
    fig.layout.yaxis.scaleanchor = "x"
    fig.update_layout(
        template="none",
        width=400,
        height=400,
        margin=dict(l=0, r=0, b=0, t=0),
        showlegend=False,
    )
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    fig.show()

## The ANN model that approximates the displacement field

An ANN might be considered as a generic function approximator. In this case, it should approximated the function $u: \mathcal{R}^2 \rightarrow \mathcal{R}^2$ with five hidden layers having 50 neurons each.

In [3]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden_layer1 = torch.nn.Linear(2, hn)
        self.hidden_layer2 = torch.nn.Linear(hn, hn)
        self.hidden_layer3 = torch.nn.Linear(hn, hn)
        self.hidden_layer4 = torch.nn.Linear(hn, hn)
        self.hidden_layer5 = torch.nn.Linear(hn, hn)
        # self.hidden_layer6 = torch.nn.Linear(hn, hn)
        # self.hidden_layer7 = torch.nn.Linear(hn, hn)
        # self.hidden_layer8 = torch.nn.Linear(hn, hn)
        self.output_layer = torch.nn.Linear(hn, 2)

    def forward(self, inputs):
        layer1_out = torch.tanh(self.hidden_layer1(inputs))
        layer2_out = torch.tanh(self.hidden_layer2(layer1_out))
        layer3_out = torch.tanh(self.hidden_layer3(layer2_out))
        layer4_out = torch.tanh(self.hidden_layer4(layer3_out))
        layer5_out = torch.tanh(self.hidden_layer5(layer4_out))
        # layer6_out = torch.tanh(self.hidden_layer5(layer5_out))
        # layer7_out = torch.tanh(self.hidden_layer5(layer6_out))
        # layer8_out = torch.tanh(self.hidden_layer5(layer7_out))
        output = self.output_layer(layer5_out)
        return output


net = Net()

## The physics

We want to solve linear elasticity on the domain, which means ultimately that we want to minimize the residual of the following PDE 
$$\frac{\partial \sigma_{11}}{\partial x_1} + \frac{\partial \sigma_{12}}{\partial x_2} - b_1 = 0$$
$$\frac{\partial \sigma_{21}}{\partial x_1} + \frac{\partial \sigma_{22}}{\partial x_2} - b_2 = 0$$
with stress 
$$ \sigma_{ij} = 2\mu \varepsilon_{ij} + \lambda \varepsilon_{kk} \delta_{ij} $$
and strain 
$$ \varepsilon_{ij} = \frac{1}{2} \left( \frac{\partial u_i}{\partial x_j} +  \frac{\partial u_j}{\partial x_i}\right).$$

In [4]:
def epsilon(x):
    # Compute deformation gradient
    dudx = torch.func.jacrev(net)(x)
    return 0.5 * (dudx + dudx.T)

def sigma(x):
    # Compute (small deformation) strain
    eps = epsilon(x)
    # Compute linear elastic strain (assuming plane strain)
    return 2.0 * MU * eps + LBD * torch.trace(eps) * torch.eye(2)


def pde_residual(x):
    # Compute stress gradient
    dsdx = torch.func.jacrev(sigma)(x)
    # Momentum balance in x direction
    residual_x = dsdx[0, 0, 0] + dsdx[0, 1, 1] - B0
    # Momentum balance in y direction
    residual_y = dsdx[1, 0, 0] + dsdx[1, 1, 1]
    return residual_x, residual_y


In [5]:
mse = torch.nn.MSELoss()


def compute_physics_losses(collocation):
    # pde
    res_x, res_y = torch.vmap(pde_residual)(collocation)
    zeros = torch.zeros_like(res_x)
    pde_error = mse((res_x), zeros) + mse((res_y), zeros)

    # left boundary
    pred_left = net(left)
    bc_left_x = EPS0 * left[:,1]
    bc_left_y = torch.zeros_like(pred_left[:,0])
    left_error = mse(pred_left[:,0], bc_left_x) + mse(pred_left[:,1], bc_left_y)

    # right boundary
    pred_right = net(right)
    bc_right_x = EPS0 * right[:,1]
    bc_right_y = torch.zeros_like(pred_right[:,0])
    right_error = mse(pred_right[:,0], bc_right_x) + mse(pred_right[:,1], bc_right_y)


    # bottom boundary
    pred_bottom = net(bottom)
    bc_bottom = torch.zeros_like(pred_bottom[:, 1])
    bottom_error = mse(pred_bottom[:, 0], bc_bottom) + mse(pred_bottom[:, 1], bc_bottom)


    # top boundary
    pred_top = net(top)
    bc_top_x = EPS0 * L * torch.ones_like(pred_top[:, 0])
    bc_top_y = torch.zeros_like(pred_top[:, 1])
    top_error = mse(pred_top[:, 0], bc_top_x) + mse(pred_top[:, 1], bc_top_y)

    return (left_error, right_error, bottom_error, top_error, pde_error)

## Training 

In [6]:
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
#old scheduler
scheduler = StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)
history = []

print("Starting with Adam optimizer...")
for epoch in tqdm(range(epochs)):
    # Permutation to shuffle collocation points randomly in each epoch
    permutation = torch.randperm(collocation.size()[0])

    for i in range(0, collocation.size()[0], batch_size):
        indices = permutation[i : i + batch_size]
        collo = collocation[indices]
        optimizer.zero_grad()

        # Compute physics losses
        left_l, right_l, bottom_l, top_l, pde_l = compute_physics_losses(collo)

        # Aggregate losses
        dirichlet_losses = left_l + right_l + bottom_l + top_l
        loss = (
            dirichlet_losses
            + W_PDE * pde_l
        )

        # Make optimization step after batch
        loss.backward(retain_graph=True)
        optimizer.step()

    # Make scheduler step after full epoch
    scheduler.step()
    
    # append loss to history (=for plotting)
    with torch.autograd.no_grad():
        history.append(float(loss.data))
        if(float(loss.data) <= 10**-10):
            break



plt.plot(history, c='g', label='train', linewidth=2.0)
plt.yscale("log")
plt.title("Training")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend()
plt.show()



Starting with Adam optimizer...


  0%|          | 0/5000 [00:00<?, ?it/s]


NameError: name 'EPS0' is not defined

## Visualization of results

In [ ]:
# Create a validation domain different from the training domain
val_x, val_y = np.meshgrid(np.linspace(0, L, 25), np.linspace(0, L, 25))
val_domain = np.vstack([val_x.ravel(), val_y.ravel()]).T
val = torch.tensor(val_domain, requires_grad=True).double()

# Compute model predictions on the validation domain
disp = net(val)
def_val = val + disp
stress = torch.vmap(sigma)(val)
mises = torch.sqrt(
    stress[:, 0, 0] ** 2
    + stress[:, 1, 1] ** 2
    - stress[:, 0, 0] * stress[:, 1, 1]
    + 3 * stress[:, 0, 1] ** 2
)
# print([loss.item() for loss in compute_les(val)])

@torch.no_grad()
def make_plot(x, y, variable, title, cmap=sequential.Viridis, size=8.0):
    fig = go.Figure()

    # Plot boundaries
    m = dict(color="black")
    fig.add_trace(go.Scatter(x=top[:, 0], y=top[:, 1], mode="lines", marker=m))
    fig.add_trace(go.Scatter(x=bottom[:, 0], y=bottom[:, 1], mode="lines", marker=m))
    fig.add_trace(go.Scatter(x=left[:, 0], y=left[:, 1], mode="lines", marker=m))
    fig.add_trace(go.Scatter(x=right[:, 0], y=right[:, 1], mode="lines", marker=m))

    # Plot variable values
    m = dict(color=variable, colorscale=cmap, size=size, colorbar=dict(thickness=10))
    fig.add_trace(go.Scatter(x=x, y=y, marker=m, mode="markers"))

    # plot settings
    fig.layout.yaxis.scaleanchor = "x"
    fig.update_layout(
        template="none", width=400, height=400, title=title, showlegend=False
    )
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    fig.show()


# Compute data error
s_pred = torch.vmap(sigma)(val)
ds_xx = s_pred[:, 0, 0]
ds_yy = s_pred[:, 1, 1]
ds_xy = torch.abs(MU*EPS0 - s_pred[:, 0, 1])/(MU*EPS0)



with torch.autograd.no_grad():
    print("MAX_value for stress_xx error is", torch.max(ds_xx))
    print("MAX_value for stress_yy error is", torch.max(ds_yy))
    print("MAX_value for stress_xy error is", torch.max(ds_xy))
    print("MEAN for stress_xx error is", torch.mean(ds_xx))
    print("MEAN for stress_yy error is", torch.mean(ds_yy))
    print("MEAN for stress_xy error is", torch.mean(ds_xy))
    ds_xx_np = ds_xx.numpy()
    ds_yy_np = ds_yy.numpy()
    ds_xy_np = ds_xy.numpy()
    print("75%-Quantile for stress_xx error is", np.quantile(ds_xx_np, .75))
    print("75%-Quantile for stress_yy error is", np.quantile(ds_yy_np, .75))
    print("75%-Quantile for stress_xy error is", np.quantile(ds_xy_np, .75))

# Plot stresses
make_plot(*def_val.T, stress[:, 0, 0], "Stress xx")
make_plot(*def_val.T, stress[:, 0, 1], "Stress xy")
make_plot(*def_val.T, stress[:, 1, 1], "Stress yy")
make_plot(*def_val.T, mises, "Mises stress")

# Plot displacements
make_plot(*def_val.T, disp[:, 0], "Displacement in x", cmap=sequential.Inferno)
make_plot(*def_val.T, disp[:, 1], "Displacement in y", cmap=sequential.Inferno)

# Plot stress errors
cmap = sequential.RdBu_r
make_plot(*val[:, 0:2].T, ds_xx, "Stress error xx", size=2.0, cmap=cmap)
make_plot(*val[:, 0:2].T, ds_yy, "Stress error yy", size=2.0, cmap=cmap)
make_plot(*val[:, 0:2].T, ds_xy, "Stress error xy in %", size=2.0, cmap=cmap)
